In [14]:
import wrds
import pandas as pd
from tqdm import tqdm

In [15]:
conn = wrds.Connection()

Enter your WRDS username [senior]: ctyy
Enter your password: ·············


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
Loading library list...
Done


In [16]:
company_narrow = conn.get_table(library='comp', table='funda', obs=5)

## 計算cash flow

In [17]:
# a = xrd b = sic
query = """
select a.ibc, a.dp, a.at, a.gvkey, a.datadate
from comp.funda a

WHERE a.gvkey = '001562'

"""

df_sql = conn.raw_sql(query, date_cols=['datadate'])
df_sql

,ibc,dp,at,gvkey,datadate
0,3.319,0.082,25.698,001562,1983-04-30
1,4.233,0.596,35.069,001562,1984-04-30
2,5.470,1.009,38.873,001562,1985-04-30
3,5.915,1.602,50.954,001562,1986-04-30
4,8.805,1.831,67.930,001562,1987-04-30
...,...,...,...,...,...
60,NaN,8.103,171.293,001562,2020-04-30
61,8.089,5.610,177.157,001562,2021-04-30
62,NaN,5.610,177.157,001562,2021-04-30
63,12.782,4.139,192.835,001562,2022-04-30


In [18]:
df_sql['cash_flow'] = (df_sql['ibc']+df_sql['dp'])/df_sql['at']
df_sql

,ibc,dp,at,gvkey,datadate,cash_flow
0,3.319,0.082,25.698,001562,1983-04-30,0.132345
1,4.233,0.596,35.069,001562,1984-04-30,0.137700
2,5.470,1.009,38.873,001562,1985-04-30,0.166671
3,5.915,1.602,50.954,001562,1986-04-30,0.147525
4,8.805,1.831,67.930,001562,1987-04-30,0.156573
...,...,...,...,...,...,...
60,NaN,8.103,171.293,001562,2020-04-30,NaN
61,8.089,5.610,177.157,001562,2021-04-30,0.077327
62,NaN,5.610,177.157,001562,2021-04-30,NaN
63,12.782,4.139,192.835,001562,2022-04-30,0.087749


In [19]:
## 取得SIC Code 737開頭的公司財務資料

In [20]:
# a = xrd b = sic
query = """
select a.at, a.ebit, a.csho, a.prcc_f, a.ceq, a.ni, a.oancf, a.opeps, a.teq, a.xrd, a.sale, c.sic, c.tic, d.conml, a.gvkey, a.datadate

from comp.funda a

inner join comp.namesq c
on a.gvkey = c.gvkey

inner join comp.company d
on a.gvkey = d.gvkey

WHERE a.datadate>='01/01/2010'
and a.datadate<='12/31/2020'
and c.sic like '737%%'
and a.xrd is not null
and a.xrd > 0
and a.ebit is not null

"""

df_sql = conn.raw_sql(query, date_cols=['datadate'])
df_sql.to_csv("comp funda.csv",index=False)
df_sql

,at,ebit,csho,prcc_f,ceq,ni,oancf,opeps,teq,xrd,sale,sic,tic,conml,gvkey,datadate
0,2447.200,134.800,228.881,23.79,1473.500,58.000,246.800,0.54,1473.500,457.500,1713.700,7370,ADSK,AUTODESK INC,001878,2010-01-31
1,97.175,7.979,25.374,6.39,72.280,5.704,-1.876,0.25,72.280,6.722,75.276,7372,AMSWA,American Software Inc,001562,2010-04-30
2,104.832,9.412,26.050,7.78,74.056,7.364,3.066,0.29,74.056,7.388,85.589,7372,AMSWA,American Software Inc,001562,2011-04-30
3,116.553,16.168,27.037,8.28,83.030,11.343,16.281,0.43,83.030,8.226,102.636,7372,AMSWA,American Software Inc,001562,2012-04-30
4,113.070,13.784,27.327,8.31,83.244,10.411,19.128,0.38,83.244,8.882,100.467,7372,AMSWA,American Software Inc,001562,2013-04-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5451,108.754,38.322,0.001,NaN,43.983,16.851,33.588,0.79,59.574,3.434,120.390,7370,KARO,Karooooo Ltd,347085,2019-02-28
5452,764.288,-185.465,45.560,NaN,-312.467,-178.028,-143.982,-0.64,598.386,68.681,96.666,7370,SNOW,Snowflake Inc,339965,2019-01-31
5453,1012.720,-358.088,55.452,NaN,-544.757,-348.535,-176.558,-1.26,391.717,105.160,264.748,7370,SNOW,Snowflake Inc,339965,2020-01-31
5454,4.813,-8.396,17.413,NaN,1.911,-10.684,-4.483,-0.83,2.074,2.742,2.648,7372,IDAI,T Stamp Inc,345764,2020-12-31
